In [44]:
from typing import TypeVar, List, Callable
import random
import enum
import numpy as np
import pandas as pd


T = TypeVar('T')

# Logic
## a)

In [34]:
test_strs = ["y", "o"]
comp_str = "Hello World"

any([x for x in test_strs if x in comp_str])

True

## b)

In [45]:
def xor(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    return np.logical_and(
        np.logical_not(
            np.logical_and(a, b)
            ),
        np.logical_or(a, b)
        )
df = pd.DataFrame(
        np.array(np.meshgrid([True, False], [True, False])).T.reshape(-1, 2),
        columns=["op1", "op2"])
df.assign(xor_result = lambda x: xor(x["op1"], x["op2"]))

,op1,op2,xor_result
0,True,True,False
1,True,False,True
2,False,True,True
3,False,False,False


# Age Category

In [6]:
# transform input and return error message if input cannot be transformed
def inputx(prompt: str, transform: Callable[[str], T] = lambda x: x) -> T:
    while True:
        try:
            return transform(input(prompt))
        except ValueError:
            from IPython.display import display, Markdown
            display(Markdown("<span style='color:red'>Invalid input. Please try again.</span>"))

def classify_age(age: int) -> str | None:
    if age < 0:
        return None
    elif age <= 2:
        return "Infant"
    elif age <= 12:
        return "Child"
    elif age <= 19:
        return "Teenager"
    elif age <= 64:
        return "Adult"
    else:
        return "Senior"

classify_age(inputx("Enter your age: ", int))

'Child'

# Number Classifier

In [13]:
# superclass for all number classifiers
class NumberClassifier():
    @staticmethod
    def classify(number: int) -> str:
        raise NotImplementedError("Subclass must implement abstract method")

class NumberSignClassifier(NumberClassifier):
    @staticmethod
    def classify(number: int) -> str:
        if number < 0:
            return "Negative"
        elif number == 0:
            return "Zero"
        else:
            return "Positive"
    
class NumberParityClassifier(NumberClassifier):
    @staticmethod
    def classify(number: int) -> str:
        if number % 2 == 0:
            return "Even"
        else:
            return "Odd"

input_number = inputx("Enter a number: ", int)
classifiers = [NumberSignClassifier, NumberParityClassifier]
print(f"Classifying {input_number}")
for classifier in classifiers:
    print(f"{classifier.__name__}: {classifier.classify(input_number)}")

Classifying 12
NumberSignClassifier: Positive
NumberParityClassifier: Even


# Calculator

In [26]:

class Expr:
    def _eval(self) -> float:
        raise NotImplementedError("Subclass must implement abstract method")
                                                                        
    def eval(self) -> float:
        return self._eval()


class NumberLiteral(Expr):
    def __init__(self, value: float):
        self.value = value

    def _eval(self) -> float:
        return self.value

    def __str__(self) -> str:
        return str(self.value)

    def __repr__(self) -> str:
        return f"NumberLiteral({self.value})"


class Unop(Expr):
    def __init__(self, operand: Expr, op: str):
        self.operand = operand
        self.op = op

    def _eval(self) -> float:
        if self.op == "+":
            return self.operand._eval()
        elif self.op == "-":
            return -self.operand._eval()
        else:
            raise ValueError(f"Invalid operator {self.op}")

    def __str__(self) -> str:
        return f"{self.op}{self.operand}"

    def __repr__(self) -> str:
        return f"Unop({self.operand}, {self.op})"


class BinOp(Expr):
    def __init__(self, left: Expr, right: Expr, op: str):
        self.left = left
        self.right = right
        self.op = op

    def _eval(self) -> float:
        if self.op == "+":
            return self.left._eval() + self.right._eval()
        elif self.op == "-":
            return self.left._eval() - self.right._eval()
        elif self.op == "*":
            return self.left._eval() * self.right._eval()
        elif self.op == "/":
            return self.left._eval() / self.right._eval()
        else:
            raise ValueError(f"Invalid operator {self.op}")

    def __str__(self) -> str:
        return f"({self.left}{self.op}{self.right})"

    def __repr__(self) -> str:
        return f"BinOp({self.left}, {self.right}, {self.op})"


class Tokenizer:
    class TokenizerState(enum.Enum):
        START = 0
        NUMBER = 1

    def __init__(self, expr: str):
        self.expr = expr
        self.pos = 0
        self.state = Tokenizer.TokenizerState.START

    def _next_token(self) -> str | None:
        if self.pos >= len(self.expr):
            if self.state == Tokenizer.TokenizerState.NUMBER:
                self.state = Tokenizer.TokenizerState.START
                return self.number
            return None
        elif self.state == Tokenizer.TokenizerState.START:
            if self.expr[self.pos] in "+-*/()":
                self.pos += 1
                return self.expr[self.pos - 1]
            elif self.expr[self.pos].isdigit():
                self.state = Tokenizer.TokenizerState.NUMBER
                self.number = ""
                return self._next_token()
            elif self.expr[self.pos].isspace():
                self.pos += 1
                return self._next_token()
            else:
                raise ValueError(f"Invalid character {self.expr[self.pos]}")
        elif self.state == Tokenizer.TokenizerState.NUMBER:
            if self.expr[self.pos].isdigit():
                self.pos += 1
                self.number += self.expr[self.pos - 1]
                return self._next_token()
            else:
                self.state = Tokenizer.TokenizerState.START
                return self.number
        else:
            raise ValueError(f"Invalid state {self.state}")

    def tokenize(self) -> List[str]:
        tokens = []
        while True:
            token = self._next_token()
            if token is None:
                break
            else:
                tokens.append(token)
        return tokens


class Parser:
    def _parse_expr(self, tokens: List[str | Expr]) -> Expr:
        if "(" in tokens:
            start = tokens.index("(")
            end = start + 1
            count = 1
            while count > 0:
                if tokens[end] == "(":
                    count += 1
                elif tokens[end] == ")":
                    count -= 1
                end += 1
            return self._parse_expr(tokens[:start] + [self._parse_expr(tokens[start + 1:end - 1])] + tokens[end:])

        if len(tokens) == 1:
            if isinstance(tokens[0], Expr):
                return tokens[0]
            if tokens[0].isdigit():
                return NumberLiteral(float(tokens[0]))
            elif tokens[0] in "+-":
                return Unop(NumberLiteral(0), tokens[0])
            else:
                raise ValueError(f"Invalid token {tokens[0]}")
        elif len(tokens) == 2:
            if tokens[0] in "+-":
                return Unop(self._parse_expr(tokens[1:]), tokens[0])
            else:
                raise ValueError(f"Invalid token {tokens[0]}")
        elif len(tokens) == 3:
            if tokens[1] in "+-*/":
                return BinOp(self._parse_expr(tokens[:1]), self._parse_expr(tokens[2:]), tokens[1])
            else:
                raise ValueError(f"Invalid token {tokens[1]}")
        else:
            if "*" in tokens or "/" in tokens:
                if "*" in tokens:
                    op = "*"
                else:
                    op = "/"
                op_index = tokens.index(op)
                return self._parse_expr(tokens[:op_index - 1] + [BinOp(self._parse_expr(tokens[op_index - 1:op_index]), self._parse_expr(tokens[op_index + 1:op_index + 2]), op)] + tokens[op_index + 2:])
            elif "+" in tokens or "-" in tokens:
                if "+" in tokens:
                    op = "+"
                else:
                    op = "-"
                op_index = tokens.index(op)
                return self._parse_expr(tokens[:op_index - 1] + [BinOp(self._parse_expr(tokens[op_index - 1:op_index]), self._parse_expr(tokens[op_index + 1:op_index + 2]), op)] + tokens[op_index + 2:])
            else:
                raise ValueError(f"Invalid tokens {tokens}")

    def parse(self, tokens: List[str]) -> Expr:
        return self._parse_expr(tokens)

expr = inputx("Enter an expression: (e.g. 1+2*3)")
print(f"Evaluating {expr}")
print(Parser().parse(Tokenizer(expr).tokenize()).eval())

Evaluating 1+2*3
7.0


# Guessing Game

In [25]:
class GuessResult(enum.Enum):
    TOO_LOW = enum.auto()
    TOO_HIGH = enum.auto()
    CORRECT = enum.auto()

    def __str__(self):
        return self.name.replace("_", " ").title()

class GuessingGame():
    def __init__(self, answer: int, max_guesses: int = 5):
        self.answer = answer
        self.max_guesses = max_guesses
        self.guesses = 0
    
    def guess(self, guess: int) -> GuessResult:
        self.guesses += 1
        if self.guesses > self.max_guesses:
            raise ValueError("Too many guesses")
        if guess < self.answer:
            return GuessResult.TOO_LOW
        elif guess > self.answer:
            return GuessResult.TOO_HIGH
        else:
            return GuessResult.CORRECT
    
    def is_game_over(self) -> bool:
        return self.guesses >= self.max_guesses
    
    def play(self) -> None:
        while not self.is_game_over():
            guess = inputx("Enter your guess: ", int)
            result = self.guess(guess)
            print(result)
            if result == GuessResult.CORRECT:
                print("You win!")
                return
        print("Game over, the answer was", self.answer)

game = GuessingGame(random.randint(1, 10))
game.play()



Too High
Too High
Too High
Correct
You win!


# Counter Contrast

In [22]:
class CounterContrast():
    @staticmethod
    def using_for_loop(to: int) -> None:
        for i in range(1, to + 1):
            print(i)
    @staticmethod
    def using_while_loop(to: int) -> None:
        i = 1
        while i <= to:
            print(i)
            i += 1

number = inputx("Enter a number: ", int)
print(f"Counting to {number}")
print("Using for loop:")
CounterContrast.using_for_loop(number)
print("Using while loop:")
CounterContrast.using_while_loop(number)

Counting to 5
Using for loop:
1
2
3
4
5
Using while loop:
1
2
3
4
5


A for loop is better because we are iterating over an iterable(range).

# Multiplication Table

In [ ]:
class MultiplicationTabls():
    @staticmethod
    def using_for_loop(x: int, to: int = 10) -> None:
        for i in range(1, to + 1):
            print(f"{x} x {i} = {x * i}")
    @staticmethod
    def using_while_loop(x: int, to: int = 10) -> None:
        i = 1
        while i <= to:
            print(f"{x} x {i} = {x * i}")
            i += 1

number = inputx("Enter a number: ", int)
MultiplicationTabls.using_for_loop(number)

5 x 1 = 5
5 x 2 = 10
5 x 3 = 15
5 x 4 = 20
5 x 5 = 25
5 x 6 = 30
5 x 7 = 35
5 x 8 = 40
5 x 9 = 45
5 x 10 = 50


A for loop is better because we are iterating over an iterable(range).

# Entering a positive number

I would use a while loop because this is looping until some condition becomes true. To use a for loop you need to write a custom iterator to achieve this. A while loop would be more concise and appropriate for this task.